In [1]:
class Txn:
    def __init__(self, name, write_set=set(), read_set=set(),should_abort=False):
        self.name = name
        self.read_set = read_set
        self.write_set = write_set
        self.should_abort = should_abort
    def __str__(self):
        return self.name
    def __repr__(self):
        return self.name

In [2]:
# Sample data
# Lock table:
# X: [A, E]
# Y: [B]
# Z: [C, D, E]
# our adjacency list should look like this:
# A: [E]
# B: []
# C: [D, E]
# D: [E]

In [3]:
txns = [
    Txn("A", {"X"}),
    Txn("B", {"Y"}),
    Txn("C", {"Z"}),
    Txn("D", {"Z"}),
    Txn("E", {"Z", "X"}),
]

In [4]:
from collections import defaultdict, deque

# 1) build the lock table
lock_table = defaultdict(deque)  # maps record -> queue of txns

for txn in txns:
    for key in txn.write_set:
        lock_table[key].append(txn)

lock_table

defaultdict(collections.deque,
            {'X': deque([A, E]), 'Y': deque([B]), 'Z': deque([C, D, E])})

In [5]:
# 2) build the dependency graph

adj = { txn: [] for txn in txns }

for key, _txns in lock_table.items():
    if len(_txns) <= 1:
        continue
    for i in range(1, len(_txns)):
        prev, cur = _txns[i - 1], _txns[i]
        adj[prev].append(cur)

print(adj)

{A: [E], B: [], C: [D], D: [E], E: []}


In [6]:
# Regular topological sort (course schedule II)

res = []
visit, cycle = set(), set()
def dfs(node):
    if node in visit:
        return True
    if node in cycle:
        return False

    cycle.add(node)

    for nei in adj[node]:
        if not dfs(nei):
            return False

    res.append(node)
    cycle.remove(node)
    visit.add(node)
    return True

for txn in txns:
    if not dfs(txn):
        print("error")
res[::-1]

[C, D, B, A, E]

In [7]:
# Kahn's algorithm

indegrees = {v: 0 for v in adj}

# O(V + E)
for txn in adj:
    for nei in adj[txn]:
        indegrees[nei] += 1

indegrees

{A: 0, B: 0, C: 0, D: 1, E: 2}

In [8]:
q = deque([])
for txn in indegrees:
    if indegrees[txn] == 0:
        q.append(txn)
q

deque([A, B, C])

In [9]:
# say we complete A
txn = q.popleft()
for nei in adj[txn]:
    indegrees[nei] -= 1
    if indegrees[nei] == 0:
        q.append(nei)

q

deque([B, C])

In [10]:
# say we complete B
txn = q.popleft()
for nei in adj[txn]:
    indegrees[nei] -= 1
    if indegrees[nei] == 0:
        q.append(nei)

q

deque([C])

In [11]:
# say we complete C
txn = q.popleft()
for nei in adj[txn]:
    indegrees[nei] -= 1
    if indegrees[nei] == 0:
        q.append(nei)

q

deque([D])

In [12]:
# say we complete D
txn = q.popleft()
for nei in adj[txn]:
    indegrees[nei] -= 1
    if indegrees[nei] == 0:
        q.append(nei)

q

deque([E])

In [13]:
# say we complete E
txn = q.popleft()
for nei in adj[txn]:
    indegrees[nei] -= 1
    if indegrees[nei] == 0:
        q.append(nei)

q

deque([])